In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [141]:
train=pd.read_csv('f:/nettrainsparse.csv',usecols=[1,2,3])
test=pd.read_csv('f:/nettestsparse.csv',usecols=[1,2,3])

In [142]:
import time

In [143]:
no_movies = 9318
no_users = 8141

In [144]:
import math

In [145]:
rated= [set() for i in range(no_movies) ]
dictionary={}

In [146]:
movies_mean=np.zeros(shape=[no_movies,3])

In [147]:
user_total=np.zeros(no_users)
cnt=np.zeros(no_users,dtype='int64')
for i, row in enumerate(train.itertuples(), 1):
    p=int(row.itemId)
    movies_mean[p][0]+=1
    movies_mean[p][1]+=row.rating
    k=int(row.userId)
    cnt[k]+=1
    user_total[k]+=row.rating

In [148]:
user_mean=np.array([ i/j for i,j in zip(user_total,cnt) ])
for x in movies_mean:
    x[2]=round(x[1]/x[0],2)
    
for x in movies_mean:
    if math.isnan(x[2]):
        x[2]=0


In [149]:
import math

In [150]:
for i in range(len(user_mean)):
    if math.isnan(user_mean[i]):
        user_mean[i]=0

In [151]:
foruser=np.zeros(no_users)
formovies=np.zeros(no_movies)
for i, row in enumerate(train.itertuples(), 1):
    mov=int(row.itemId)
    usr=int(row.userId)
    foruser[usr]+=movies_mean[mov][2]
    formovies[mov]+=user_mean[usr]

In [152]:
deviation=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)
cardinality=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)

In [153]:
tendency=np.zeros(no_movies)

In [154]:
for i in range(no_movies):
    if movies_mean[i][0]!=0:
        tendency[i]=(movies_mean[i][1]-formovies[i])/(movies_mean[i][0])

In [155]:
rmse_normal=0
mae_normal=0
gim=0
gpim=0
tp=0
fp=0
tn=0
fn=0
c_gim=0
c_gpim=0
for i, row in enumerate(test.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.itemId)
    actual=row.rating
    pred=0
    list_of_all_movies_rate_by_the_current_user=train[train['userId']==usr]
    length=list_of_all_movies_rate_by_the_current_user.shape[0]
    cnt=0
    for j,r2 in enumerate(list_of_all_movies_rate_by_the_current_user.itertuples(),1):
        curmov=r2.itemId
        currate=r2.rating
        pred+=(currate+tendency[mov]-tendency[curmov])
        cnt+=1
    if cnt!=0:
        predicted_normal=(pred/cnt)
    else:
        predicted_normal=0
    if predicted_normal>5:
        predicted_normal=5
    if predicted_normal<0:
        predicted_normal=0
    if actual >=3:
        gim+=abs(actual-predicted_normal)
        c_gim+=1
    if predicted_normal >=3:
        gpim+=abs(actual-predicted_normal)
        c_gpim+=1
    
    if actual>=3 and predicted_normal>=3:
        tp+=1
    elif actual<3 and predicted_normal>=3:
        fp+=1
    elif actual>=3 and predicted_normal<3:
        fn+=1
    else:
        tn+=1
    
    rmse_normal+=((actual-predicted_normal)**2)
    mae_normal+=abs(actual-predicted_normal)
    

In [156]:
index=test['userId'].count()

In [157]:
print(mae_normal/index)

0.734152357078


In [158]:
print(math.sqrt(rmse_normal/index))

0.9348555757305614


In [159]:
gim/c_gim

0.60478982416324356

In [160]:
gpim/c_gpim

0.72502086563997215

In [161]:
tp/(tp+fp)

0.8926521568499198

In [162]:
tp/(tp+fn)

0.9096464844238188